<a href="https://colab.research.google.com/github/EmmaHsueh/PL_project/blob/main/%EF%BC%A8%EF%BC%B75.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio folium requests pandas geopy

In [2]:
import gradio as gr
import folium
from folium import plugins
import requests
import pandas as pd
from datetime import datetime
import json
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time


In [3]:
class ParkingMapSystem:
    def __init__(self):
        self.taipei_data = []
        self.newtaipei_data = []
        self.all_data = []
        self.geolocator = Nominatim(user_agent="parking_finder")

    def fetch_taipei_parking(self):
        """爬取台北市停車場資料"""
        try:
            # 台北市停車場即時資訊API
            url = "https://tcgbusfs.blob.core.windows.net/blobtcmsv/TCMSV_alldesc.json"
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()

            parking_list = []
            for item in data.get('data', {}).get('park', []):
                try:
                    parking_info = {
                        'id': item.get('id', ''),
                        'name': item.get('name', '未命名停車場'),
                        'address': item.get('address', ''),
                        'lat': float(item.get('tw97y', 0)) if item.get('tw97y') else 25.0330,
                        'lon': float(item.get('tw97x', 0)) if item.get('tw97x') else 121.5654,
                        'total_spaces': int(item.get('totalcar', 0)),
                        'available_spaces': int(item.get('availablecar', 0)),
                        'rate': item.get('payex', '資訊未提供'),
                        'hours': item.get('serviceTime', '24小時'),
                        'district': self.extract_district(item.get('address', ''), '台北市'),
                        'city': '台北市',
                        'update_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    }
                    parking_list.append(parking_info)
                except (ValueError, TypeError) as e:
                    continue

            return parking_list
        except Exception as e:
            print(f"台北市資料爬取錯誤: {e}")
            return []

    def fetch_newtaipei_parking(self):
        """爬取新北市停車場資料"""
        try:
            # 新北市停車場即時資訊API
            url = "https://data.ntpc.gov.tw/api/datasets/0EDA7713-087E-4ABD-ADAD-81F592CFCD0A/json"
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()

            parking_list = []
            for item in data:
                try:
                    available = int(item.get('availableSpace', 0))
                    total = int(item.get('totalSpace', 0))

                    parking_info = {
                        'id': item.get('parkId', ''),
                        'name': item.get('parkName', '未命名停車場'),
                        'address': item.get('address', ''),
                        'lat': float(item.get('wgsY', 0)) if item.get('wgsY') else 25.0118,
                        'lon': float(item.get('wgsX', 0)) if item.get('wgsX') else 121.4653,
                        'total_spaces': total,
                        'available_spaces': available,
                        'rate': item.get('fare', '資訊未提供'),
                        'hours': item.get('openTime', '24小時'),
                        'district': self.extract_district(item.get('address', ''), '新北市'),
                        'city': '新北市',
                        'update_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    }
                    parking_list.append(parking_info)
                except (ValueError, TypeError) as e:
                    continue

            return parking_list
        except Exception as e:
            print(f"新北市資料爬取錯誤: {e}")
            return []

    def extract_district(self, address, city):
        """從地址提取行政區"""
        if not address:
            return "未知"

        # 移除縣市名稱
        address = address.replace(city, '')

        # 常見行政區關鍵字
        districts = ['中正區', '大同區', '中山區', '松山區', '大安區', '萬華區',
                    '信義區', '士林區', '北投區', '內湖區', '南港區', '文山區',
                    '板橋區', '三重區', '中和區', '永和區', '新莊區', '新店區',
                    '樹林區', '鶯歌區', '三峽區', '淡水區', '汐止區', '瑞芳區',
                    '土城區', '蘆洲區', '五股區', '泰山區', '林口區', '深坑區',
                    '石碇區', '坪林區', '三芝區', '石門區', '八里區', '平溪區',
                    '雙溪區', '貢寮區', '金山區', '萬里區', '烏來區']

        for district in districts:
            if district in address:
                return district

        return "未知"

    def update_all_data(self):
        """更新所有停車場資料"""
        print("開始更新停車場資料...")
        self.taipei_data = self.fetch_taipei_parking()
        self.newtaipei_data = self.fetch_newtaipei_parking()
        self.all_data = self.taipei_data + self.newtaipei_data
        print(f"資料更新完成！台北市: {len(self.taipei_data)} 筆，新北市: {len(self.newtaipei_data)} 筆")
        return self.all_data

    def filter_data(self, city, district, availability):
        """篩選停車場資料"""
        filtered = self.all_data.copy()

        # 依縣市篩選
        if city != "全部":
            filtered = [p for p in filtered if p['city'] == city]

        # 依行政區篩選
        if district != "全部":
            filtered = [p for p in filtered if p['district'] == district]

        # 依空位狀態篩選
        if availability == "僅顯示有空位":
            filtered = [p for p in filtered if p['available_spaces'] > 0]
        elif availability == "僅顯示無空位":
            filtered = [p for p in filtered if p['available_spaces'] == 0]

        return filtered

    def search_by_address(self, address, data):
        """根據地址搜尋附近停車場"""
        if not address or address.strip() == "":
            return data, None

        try:
            # 使用地理編碼獲取座標
            location = self.geolocator.geocode(address + ", 台灣", timeout=10)
            if location:
                target_coords = (location.latitude, location.longitude)

                # 計算每個停車場與目標地址的距離
                for parking in data:
                    parking_coords = (parking['lat'], parking['lon'])
                    parking['distance'] = geodesic(target_coords, parking_coords).kilometers

                # 依距離排序
                data.sort(key=lambda x: x.get('distance', float('inf')))

                return data[:50], target_coords  # 只顯示最近的50個
            else:
                return data, None
        except Exception as e:
            print(f"地址搜尋錯誤: {e}")
            return data, None

    def get_marker_color(self, available, total):
        """根據剩餘車位比例決定標記顏色"""
        if total == 0:
            return 'gray'

        ratio = available / total
        if available == 0:
            return 'gray'
        elif ratio > 0.3:
            return 'green'
        elif ratio > 0.1:
            return 'orange'
        else:
            return 'red'

    def create_map(self, data, center_coords=None):
        """建立 Folium 地圖"""
        # 設定地圖中心點
        if center_coords:
            map_center = [center_coords[0], center_coords[1]]
        elif data:
            map_center = [sum(p['lat'] for p in data) / len(data),
                         sum(p['lon'] for p in data) / len(data)]
        else:
            map_center = [25.0330, 121.5654]  # 台北市政府

        # 建立地圖
        m = folium.Map(
            location=map_center,
            zoom_start=13,
            tiles='OpenStreetMap'
        )

        # 如果有搜尋中心點，標記出來
        if center_coords:
            folium.Marker(
                location=[center_coords[0], center_coords[1]],
                popup="搜尋位置",
                icon=folium.Icon(color='blue', icon='search', prefix='fa')
            ).add_to(m)

        # 加入停車場標記
        for parking in data:
            color = self.get_marker_color(parking['available_spaces'], parking['total_spaces'])

            # 計算使用率
            usage_rate = 0
            if parking['total_spaces'] > 0:
                usage_rate = ((parking['total_spaces'] - parking['available_spaces']) /
                            parking['total_spaces'] * 100)

            # 建立彈窗內容
            distance_info = ""
            if 'distance' in parking:
                distance_info = f"<br><b>📏 距離:</b> {parking['distance']:.2f} 公里"

            popup_html = f"""
            <div style="font-family: Arial; width: 250px;">
                <h4 style="margin: 0 0 10px 0; color: #2c3e50;">{parking['name']}</h4>
                <hr style="margin: 5px 0;">
                <p style="margin: 5px 0;"><b>📍 地址:</b> {parking['address']}</p>
                <p style="margin: 5px 0;"><b>🏘️ 行政區:</b> {parking['district']}</p>
                <p style="margin: 5px 0;"><b>🚗 剩餘車位:</b>
                    <span style="color: {'green' if parking['available_spaces'] > 0 else 'red'}; font-weight: bold;">
                        {parking['available_spaces']}/{parking['total_spaces']}
                    </span>
                </p>
                <p style="margin: 5px 0;"><b>📊 使用率:</b> {usage_rate:.1f}%</p>
                <p style="margin: 5px 0;"><b>💰 收費:</b> {parking['rate']}</p>
                <p style="margin: 5px 0;"><b>⏰ 營業:</b> {parking['hours']}</p>
                <p style="margin: 5px 0;"><b>🕐 更新:</b> {parking['update_time']}</p>
                {distance_info}
            </div>
            """

            folium.Marker(
                location=[parking['lat'], parking['lon']],
                popup=folium.Popup(popup_html, max_width=300),
                tooltip=f"{parking['name']} ({parking['available_spaces']}個空位)",
                icon=folium.Icon(color=color, icon='car', prefix='fa')
            ).add_to(m)

        # 加入圖層控制
        folium.LayerControl().add_to(m)

        return m

    def get_districts(self, city):
        """根據選擇的縣市回傳行政區列表"""
        if city == "台北市":
            return ["全部", "中正區", "大同區", "中山區", "松山區", "大安區", "萬華區",
                   "信義區", "士林區", "北投區", "內湖區", "南港區", "文山區"]
        elif city == "新北市":
            return ["全部", "板橋區", "三重區", "中和區", "永和區", "新莊區", "新店區",
                   "樹林區", "鶯歌區", "三峽區", "淡水區", "汐止區", "瑞芳區",
                   "土城區", "蘆洲區", "五股區", "泰山區", "林口區"]
        else:
            return ["全部"]


In [4]:
# 初始化系統
system = ParkingMapSystem()

def update_districts(city):
    """更新行政區下拉選單"""
    return gr.Dropdown(choices=system.get_districts(city), value="全部")

def update_map(city, district, availability, address):
    """更新地圖顯示"""
    try:
        # 更新資料
        if not system.all_data:
            system.update_all_data()

        # 篩選資料
        filtered_data = system.filter_data(city, district, availability)

        # 地址搜尋
        center_coords = None
        if address and address.strip():
            filtered_data, center_coords = system.search_by_address(address, filtered_data)

        # 建立地圖
        m = system.create_map(filtered_data, center_coords)

        # 統計資訊
        total_parking = len(filtered_data)
        available_parking = len([p for p in filtered_data if p['available_spaces'] > 0])
        total_spaces = sum(p['available_spaces'] for p in filtered_data)

        stats = f"""
        ### 📊 統計資訊
        - **總停車場數:** {total_parking} 個
        - **有空位停車場:** {available_parking} 個
        - **總剩餘車位:** {total_spaces} 個
        - **更新時間:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        """

        return m._repr_html_(), stats

    except Exception as e:
        error_msg = f"錯誤: {str(e)}"
        return f"<p style='color: red;'>{error_msg}</p>", error_msg


In [5]:
def refresh_data(city, district, availability, address):
    """重新爬取資料並更新地圖"""
    system.update_all_data()
    return update_map(city, district, availability, address)


In [6]:
# 建立 Gradio 介面
with gr.Blocks(title="停車場即時資訊地圖", theme=gr.themes.Soft()) as app:
    gr.Markdown(
        """
        # 🚗 停車場即時資訊地圖
        ### 提供台北市與新北市停車場即時空位查詢

        **圖示說明:**
        - 🟢 綠色：充足空位 (>30%)
        - 🟠 橘色：少量空位 (10-30%)
        - 🔴 紅色：稀少空位 (<10%)
        - ⚪ 灰色：已滿位
        - 🔵 藍色：搜尋位置
        """
    )

    with gr.Row():
        city_dropdown = gr.Dropdown(
            choices=["全部", "台北市", "新北市"],
            value="全部",
            label="選擇縣市"
        )
        district_dropdown = gr.Dropdown(
            choices=["全部"],
            value="全部",
            label="選擇行政區"
        )
        availability_radio = gr.Radio(
            choices=["顯示全部", "僅顯示有空位", "僅顯示無空位"],
            value="顯示全部",
            label="空位篩選"
        )

    with gr.Row():
        address_input = gr.Textbox(
            label="地址搜尋",
            placeholder="輸入地址搜尋附近停車場（例如：台北101）",
            scale=3
        )
        search_btn = gr.Button("🔍 搜尋", scale=1)
        refresh_btn = gr.Button("🔄 更新資料", scale=1)

    map_html = gr.HTML(label="停車場地圖")
    stats_text = gr.Markdown()

    # 事件綁定
    city_dropdown.change(
        fn=update_districts,
        inputs=[city_dropdown],
        outputs=[district_dropdown]
    )

    search_btn.click(
        fn=update_map,
        inputs=[city_dropdown, district_dropdown, availability_radio, address_input],
        outputs=[map_html, stats_text]
    )

    refresh_btn.click(
        fn=refresh_data,
        inputs=[city_dropdown, district_dropdown, availability_radio, address_input],
        outputs=[map_html, stats_text]
    )

    # 下拉選單變更時自動更新
    city_dropdown.change(
        fn=update_map,
        inputs=[city_dropdown, district_dropdown, availability_radio, address_input],
        outputs=[map_html, stats_text]
    )

    district_dropdown.change(
        fn=update_map,
        inputs=[city_dropdown, district_dropdown, availability_radio, address_input],
        outputs=[map_html, stats_text]
    )

    availability_radio.change(
        fn=update_map,
        inputs=[city_dropdown, district_dropdown, availability_radio, address_input],
        outputs=[map_html, stats_text]
    )

    # 頁面載入時自動更新
    app.load(
        fn=update_map,
        inputs=[city_dropdown, district_dropdown, availability_radio, address_input],
        outputs=[map_html, stats_text]
    )



In [ ]:
# 啟動應用
if __name__ == "__main__":
    app.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c59aa7e851eb835494.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


開始更新停車場資料...
新北市資料爬取錯誤: Expecting value: line 1 column 1 (char 0)
資料更新完成！台北市: 873 筆，新北市: 0 筆
